### Coding a sampling loop

Here, we want to write a sampling loop using the `Model` class. This will take us into the intrinsics of `tensorflow`, with its `Session` object that requires a `feed_dict` to evaluate nodes. Moreover, we will understand that `Model` is essentially just a bit init() function and a dictionary to get at the initiated nodes.

In [ ]:
from TATi.model import Model as tati

So, let's go through this a bit quicker.

#### Instantiate options

In [ ]:
from TATi.options.pythonoptions import PythonOptions

options = PythonOptions()
options.set_options(
    batch_data_files=["dataset-twoclusters.csv"],
    fix_parameters="output/biases/Variable:0=-0.045677684",
    friction_constant=1.,
    inverse_temperature=10.,
    output_activation="linear",
    loss="mean_squared",
    max_steps=100,
    parse_parameters_file="training.csv",
    parse_steps=[100],
    sampler="BAOAB",
    seed=426,
    step_width=0.1)

#### Instantiate `Model`

In [ ]:
nn = tati(options)
nn.init_input_pipeline()
nn.init_network(None, setup="sample")

# reset dataset to set its "iterator" to start
nn.reset_dataset()

#### The loop

Before we create the loop, we need to fill a `feed_dict`. This is a dictionary with values for every `tensorflow.placeholder` node. As the name suggest, these are placeholders for values fed by the user.

In [ ]:
feed_dict = {
    nn.state.nn[0].placeholder_nodes["friction_constant"]: options.friction_constant,
    nn.state.nn[0].placeholder_nodes["inverse_temperature"]: options.inverse_temperature,
    nn.state.nn[0].placeholder_nodes["step_width"]: options.step_width,
}

Moreover, we gather a list of nodes which we want to evaluate per step. Among them is `sample_step`, that triggers the a single update step as implemented by the *BAOAB* sampler.

In [ ]:
step_nodes = [nn.state.nn[0].get(item) for item in ["sample_step", "accuracy", "global_step", "loss"]]

Next, we loop for `max_steps` steps: In the loop we evaluate a specific node,  For this, however, we need the feed_dict contain the batch's features and labels.


In [ ]:
for i in range(options.max_steps):
    # place the batch inside feed_dict
    features, labels = nn.state.input_pipeline.next_batch(nn.sess)
    feed_dict.update({
        nn.state.xinput: features,
        nn.state.true_labels: labels
    })
    
    # perform the sampling step
    _, acc, step, loss = nn.sess.run(step_nodes, feed_dict=feed_dict)
    
    # print loss
    print(step, loss)

### Summary

- `Model` captures its state in the `ModelState`. In that state there is an instance of `NeuralNetwork`, one per walker.
- sampling is triggered by evaluating a specific node `sample_step`.
- evaluation of nodes requires a `Session` object and for each required `tensorflow.placeholder` an entry in a so-called `feed_dict`.